# Importing libraries

In [2]:
# Importing Libraries
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Loading Data

In [3]:
data=keras.datasets.fashion_mnist

# Getting the Data
(X_train, y_train), (X_test, y_test)=data.load_data()

In [5]:
# Normalizing the Dataset
X_train=X_train/255
X_test=X_test/255

In [6]:
# Creating Validation and Test set
X_val=X_test[:5000]
y_val=y_test[:5000]

X_test=X_test[5000:]
y_test=y_test[5000:]

In [7]:
# Creating Un-labeled dataset
X_unlabeled=X_train[10000:]

# Creating Labeled dataset
X_labeled=X_train[:10000]
y_labeled=y_train[:10000]

#Preprocessing Data and creating a validation set

In [10]:
# X_train_0 dataset will contain images rotated by 0 degrees(No rotation)
X_train_0=X_unlabeled.copy()

# X_train_90 dataset will contain images rotated by 90 degrees
X_train_90=np.rot90(X_unlabeled, axes=(1,2))

# X_train_180 dataset will contain images rotated by 180 degrees
X_train_180=np.rot90(X_unlabeled, 2, axes=(1,2))

# X_train_270 dataset will contain images rotated by 270 degrees
X_train_270=np.rot90(X_unlabeled, 3, axes=(1,2))

In [16]:
# Assigning pseudo-labels to rotated image datasets
y_train_0=np.full((50000), 0)
y_train_90=np.full((50000), 1)
y_train_180=np.full((50000), 2)
y_train_270=np.full((50000), 3)

In [17]:
# Concatenating Datasets
X_train_unlabeled_full=np.concatenate((X_train_0, X_train_90, X_train_180, X_train_270), axis=0)
y_train_unlabeled_full=np.concatenate((y_train_0, y_train_90, y_train_180, y_train_270), axis=0)

In [21]:
# The function will distribute the samples uniformly over dataset
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

In [25]:
# Randomly shuffling the concatenated dataset
X_train_unlabeled_full_shuffled, y_train_unlabeled_full_shuffled = unison_shuffled_copies(X_train_unlabeled_full, y_train_unlabeled_full)

# Creating a CNN and training it on rotations of image

In [26]:
# Creating a Convolutional Neural Network
model = keras.models.Sequential([
        keras.layers.Conv2D(64, 7, activation="relu", padding="same",
                            input_shape=[28, 28, 1]),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(128, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.Conv2D(256, 3, activation="relu", padding="same"),
        keras.layers.MaxPooling2D(2),
        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(64, activation="relu"),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(4, activation="softmax")
])

In [27]:
# Compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

In [28]:
# Creating Validation and Test Dataset for Pretext Task
X_rot_val, X_rot_train = X_train_unlabeled_full_shuffled[:10000], X_train_unlabeled_full_shuffled[10000:]
y_rot_val, y_rot_train = y_train_unlabeled_full_shuffled[:10000], y_train_unlabeled_full_shuffled[10000:]

In [29]:
# Reshaping the Inputs
X_rot_val=X_rot_val.reshape(-1, 28, 28, 1)
X_rot_train=X_rot_train.reshape(-1, 28, 28, 1)

In [31]:
tf.config.run_functions_eagerly(True)

In [ ]:
# Training the model on Pretext Task
history = model.fit(X_rot_train, y_rot_train, epochs=5, validation_data=(X_rot_val, y_rot_val))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3704: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable.debug_mode()`.
  "Even though the `tf.config.experimental_run_functions_eagerly` "


Epoch 1/5
5938/5938 [==============================] - 2536s 427ms/step - loss: 0.5245 - accuracy: 0.7892 - val_loss: 0.1712 - val_accuracy: 0.9387
Epoch 2/5
5938/5938 [==============================] - 2585s 435ms/step - loss: 0.1811 - accuracy: 0.9432 - val_loss: 0.1017 - val_accuracy: 0.9666
Epoch 3/5
5938/5938 [==============================] - 2561s 431ms/step - loss: 0.1203 - accuracy: 0.9629 - val_loss: 0.0870 - val_accuracy: 0.9719
Epoch 4/5
5601/5938 [===========================>..] - ETA: 2:25 - loss: 0.0941 - accuracy: 0.9713

In [ ]:
# Checking the Model Architecture
model.summary()

In [10]:
# Removing the top layer and addding a new top layer
model.pop()
model.add(keras.layers.Dense(10, name='dense_3', activation='softmax'))

In [12]:
# Checking the changes in Model Architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 64)        3200      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 256)         295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 256)         5

In [13]:
# Check the 'Trainable' status of each layer
for layer in model.layers:
    print(layer.name, layer.trainable)

conv2d True
max_pooling2d True
conv2d_1 True
conv2d_2 True
max_pooling2d_1 True
conv2d_3 True
conv2d_4 True
max_pooling2d_2 True
flatten True
dense True
dropout True
dense_1 True
dropout_1 True
dense_3 True


In [14]:
# Freezing the Convolutional Layers while keeping Dense layers as Trainable
for layer in model.layers:
    if layer.name in ['dense_3', 'dense_1', 'dense', 'dropout', 'dropout_1']:
      layer.trainable=True
    else:
      layer.trainable=False

In [15]:
# Checking if the changes in 'Trainable' status of each layer have taken place
for layer in model.layers:
    print(layer.name, layer.trainable)

conv2d False
max_pooling2d False
conv2d_1 False
conv2d_2 False
max_pooling2d_1 False
conv2d_3 False
conv2d_4 False
max_pooling2d_2 False
flatten False
dense True
dropout True
dense_1 True
dropout_1 True
dense_3 True


In [16]:
# Reshaping the Inputs
X_labeled=X_labeled.reshape(-1, 28, 28, 1)
X_val=X_val.reshape(-1, 28, 28, 1)
X_test=X_test.reshape(-1, 28, 28, 1)

In [17]:
# Compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

# Training the model on Downstream Task
history=model.fit(X_labeled, y_labeled, validation_data=(X_val, y_val), epochs=10)

Epoch 1/10
313/313 [==============================] - 51s 160ms/step - loss: 1.8619 - accuracy: 0.3328 - val_loss: 1.1464 - val_accuracy: 0.6266
Epoch 2/10
313/313 [==============================] - 50s 159ms/step - loss: 1.2749 - accuracy: 0.5353 - val_loss: 0.8314 - val_accuracy: 0.7446
Epoch 3/10
313/313 [==============================] - 52s 165ms/step - loss: 1.0299 - accuracy: 0.6200 - val_loss: 0.6737 - val_accuracy: 0.7644
Epoch 4/10
313/313 [==============================] - 51s 163ms/step - loss: 0.8902 - accuracy: 0.6725 - val_loss: 0.6141 - val_accuracy: 0.7880
Epoch 5/10
313/313 [==============================] - 50s 160ms/step - loss: 0.8132 - accuracy: 0.6972 - val_loss: 0.5685 - val_accuracy: 0.7916
Epoch 6/10
313/313 [==============================] - 50s 159ms/step - loss: 0.7473 - accuracy: 0.7268 - val_loss: 0.5347 - val_accuracy: 0.8030
Epoch 7/10
313/313 [==============================] - 51s 163ms/step - loss: 0.7006 - accuracy: 0.7434 - val_loss: 0.5190 - val_ac

In [60]:
# Evaluating the model on the Test set
model.evaluate(X_test, y_test)

157/157 [==============================] - 10s 65ms/step - loss: 0.4387 - accuracy: 0.8432


[0.43870440125465393, 0.8432000279426575]